In [ ]:
  !pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression

#**Procesar y crear un modelo de predicción para el dataset de german_credit con pyspark**
---


In [ ]:
df_german = spark.read.csv("/content/german_credit_data.csv", inferSchema = True, header = True)

In [ ]:
df_german.show()

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+----+
|_c0|Age|   Sex|Job|Housing|Saving accounts|Checking account|Credit amount|Duration|            Purpose|Risk|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+----+
|  0| 67|  male|  2|    own|             NA|          little|         1169|       6|           radio/TV|good|
|  1| 22|female|  2|    own|         little|        moderate|         5951|      48|           radio/TV| bad|
|  2| 49|  male|  1|    own|         little|              NA|         2096|      12|          education|good|
|  3| 45|  male|  2|   free|         little|          little|         7882|      42|furniture/equipment|good|
|  4| 53|  male|  2|   free|         little|          little|         4870|      24|                car| bad|
|  5| 35|  male|  1|   free|             NA|              NA|         9055|      36|          education|good|
|  6| 53| 

In [ ]:
indexer = StringIndexer(inputCols=["Sex","Housing","Saving accounts","Checking account","Purpose","Risk"], outputCols= ["g_Sex","g_Housing","g_Saving accounts","g_Checking account","g_Purpose","g_Risk"])


In [ ]:
indexed = indexer.fit(df_german).transform(df_german)
indexed.show()

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+----+-----+---------+-----------------+------------------+---------+------+
|_c0|Age|   Sex|Job|Housing|Saving accounts|Checking account|Credit amount|Duration|            Purpose|Risk|g_Sex|g_Housing|g_Saving accounts|g_Checking account|g_Purpose|g_Risk|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+----+-----+---------+-----------------+------------------+---------+------+
|  0| 67|  male|  2|    own|             NA|          little|         1169|       6|           radio/TV|good|  0.0|      0.0|              1.0|               1.0|      1.0|   0.0|
|  1| 22|female|  2|    own|         little|        moderate|         5951|      48|           radio/TV| bad|  1.0|      0.0|              0.0|               2.0|      1.0|   1.0|
|  2| 49|  male|  1|    own|         little|              NA|         2096|      12|          educat

In [ ]:
vector = VectorAssembler(inputCols=["Age","g_Sex","Job","g_Housing","g_Saving accounts","g_Checking account","Credit amount","Duration","g_Purpose"], outputCol="features")

In [ ]:
df_vectorsito = vector.transform(indexed)

In [ ]:
logis_model = LogisticRegression(labelCol="g_Risk")

In [ ]:
model_fit = logis_model.fit(df_vectorsito)


In [ ]:
model_fit.transform(df_vectorsito).show()

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+----+-----+---------+-----------------+------------------+---------+------+--------------------+--------------------+--------------------+----------+
|_c0|Age|   Sex|Job|Housing|Saving accounts|Checking account|Credit amount|Duration|            Purpose|Risk|g_Sex|g_Housing|g_Saving accounts|g_Checking account|g_Purpose|g_Risk|            features|       rawPrediction|         probability|prediction|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+----+-----+---------+-----------------+------------------+---------+------+--------------------+--------------------+--------------------+----------+
|  0| 67|  male|  2|    own|             NA|          little|         1169|       6|           radio/TV|good|  0.0|      0.0|              1.0|               1.0|      1.0|   0.0|[67.0,0.0,2.0,0.0...|[2.41717839481594...|[0.91812789720087